In [3]:
type Language = {
    languageName: string
    level: number
}

type Interpreter = {
    languages: Language[]
}

function makeInterpreter(languages: Record<string, number>): Interpreter {
    return { languages: Object.entries(languages).map(([k, v]) => { return {languageName: k, level: v} }) }
}
                                  
const EXAMPLE_DATA = [
    { italian: 3},
    { french: 3 },
    { 'french (québécois)': 1, italian: 2}
].map(makeInterpreter)

function logExample(): void {
    console.log(JSON.stringify(EXAMPLE_DATA, null, 2))
}

In [4]:
function show(data: any) {
    return JSON.stringify(data, null, 2)
}

show(EXAMPLE_DATA)

[
  {
    "languages": [
      {
        "languageName": "italian",
        "level": 3
      }
    ]
  },
  {
    "languages": [
      {
        "languageName": "french",
        "level": 3
      }
    ]
  },
  {
    "languages": [
      {
        "languageName": "french (québécois)",
        "level": 1
      },
      {
        "languageName": "italian",
        "level": 2
      }
    ]
  }
]


In [6]:
const languageArraySort = (language: string, subField: string = 'level') => {
  return (a: Interpreter, b: Interpreter): number => {
    const aLanguages = a.languages.map((f: Language) => f.languageName)
    const bLanguages = b.languages.map((f: Language) => f.languageName)
    let aI = aLanguages.indexOf(language)
    let bI = bLanguages.indexOf(language)
    if (aI === -1) {
      const match = aLanguages.find((al: string) => al.includes(language))
      aI = aLanguages.indexOf(match)
    }
    if (bI === -1) {
      const match = bLanguages.find((bl: string) => bl.includes(language))
      bI = bLanguages.indexOf(match)
    }
    return a.languages[aI === -1 ? 0 : aI][subField] === b.languages[bI === -1 ? 0 : bI][subField]
      ? 0
      : a.languages[aI === -1 ? 0 : aI][subField] > b.languages[bI === -1 ? 0 : bI][subField] ? 1 : -1
  }
}

In [7]:
show(EXAMPLE_DATA.slice().sort(languageArraySort('french')))

[
  {
    "languages": [
      {
        "languageName": "french (québécois)",
        "level": 1
      },
      {
        "languageName": "italian",
        "level": 2
      }
    ]
  },
  {
    "languages": [
      {
        "languageName": "italian",
        "level": 3
      }
    ]
  },
  {
    "languages": [
      {
        "languageName": "french",
        "level": 3
      }
    ]
  }
]


In [8]:
const languageArraySort2 = (language: string, subField: string = 'level') => {
  return (a: Interpreter, b: Interpreter): number => {
    function languageLevel(i: Interpreter): number | undefined {
        return i
            .languages
            .find(l => l.languageName.includes(language))
            ?.level
    } 
      
    const aLevel: number | undefined = languageLevel(a)
    const bLevel: number | undefined = languageLevel(b)
    
    if (aLevel === undefined) {
        if (bLevel === undefined) {
            return 0
        } else {
            return 1
        }
    } else {
        if (bLevel === undefined) {
            return -1
        } else {
            return aLevel - bLevel
        }
    }
  }
}

In [9]:
show(EXAMPLE_DATA.slice().sort(languageArraySort2('french')))

[
  {
    "languages": [
      {
        "languageName": "french (québécois)",
        "level": 1
      },
      {
        "languageName": "italian",
        "level": 2
      }
    ]
  },
  {
    "languages": [
      {
        "languageName": "french",
        "level": 3
      }
    ]
  },
  {
    "languages": [
      {
        "languageName": "italian",
        "level": 3
      }
    ]
  }
]


In [10]:
import _ from 'lodash'

In [11]:
const exactLanguageLevel = (language: string) => (i: Interpreter) => i.languages.find(l => l.languageName === (language))?.level
const approxLanguageLevel = (language: string) => (i: Interpreter) => i.languages.find(l => l.languageName.includes(language))?.level

show(_.sortBy(EXAMPLE_DATA.slice(), [exactLanguageLevel('french'), approxLanguageLevel('french')]))

[
  {
    "languages": [
      {
        "languageName": "french",
        "level": 3
      }
    ]
  },
  {
    "languages": [
      {
        "languageName": "french (québécois)",
        "level": 1
      },
      {
        "languageName": "italian",
        "level": 2
      }
    ]
  },
  {
    "languages": [
      {
        "languageName": "italian",
        "level": 3
      }
    ]
  }
]


In [12]:
class Ordering<T> {
    readonly run: (left: T, right: T) => number
    
    constructor(run: (left: T, right: T) => number) {
        this.run = run
    }
    
    then(next: Ordering<T>): Ordering<T> {
        return new Ordering(
            (left: T, right: T) => {
                const thisOrder = this.run(left, right)
                if (thisOrder === 0) {
                    return next.run(left, right)
                } else {
                    return thisOrder
                }
            }
        )
    }
}

In [13]:
function compareExactLanguage(language: string): Ordering<Interpreter> {
    return new Ordering((a, b) => {
        function languageLevel(i: any): number | undefined {
            return i
                .languages
                .find(l => l.languageName === language)
                ?.level
        } 

        const aLevel: number | undefined = languageLevel(a)
        const bLevel: number | undefined = languageLevel(b)

        if (aLevel === undefined) {
            if (bLevel === undefined) {
                return 0
            } else {
                return 1
            }
        } else {
            if (bLevel === undefined) {
                return -1
            } else {
                return aLevel - bLevel
            }
        } 
    })
}

In [14]:
function compareApproxLanguage(language: string): Ordering<Interpreter> {
    return new Ordering((a, b) => {
        function languageLevel(i: any): number | undefined {
            return i
                .languages
                .find(l => l.languageName.includes(language))
                ?.level
        } 

        const aLevel: number | undefined = languageLevel(a)
        const bLevel: number | undefined = languageLevel(b)

        if (aLevel === undefined) {
            if (bLevel === undefined) {
                return 0
            } else {
                return 1
            }
        } else {
            if (bLevel === undefined) {
                return -1
            } else {
                return aLevel - bLevel
            }
        } 
    })
}

In [15]:
show(EXAMPLE_DATA.slice().sort(compareExactLanguage('french').then(compareApproxLanguage('french')).run))

[
  {
    "languages": [
      {
        "languageName": "french",
        "level": 3
      }
    ]
  },
  {
    "languages": [
      {
        "languageName": "french (québécois)",
        "level": 1
      },
      {
        "languageName": "italian",
        "level": 2
      }
    ]
  },
  {
    "languages": [
      {
        "languageName": "italian",
        "level": 3
      }
    ]
  }
]


In [16]:
class Ordering<T> {
    readonly run: (left: T, right: T) => number
    
    constructor(run: (left: T, right: T) => number) {
        this.run = run
    }
    
    then(next: Ordering<T>): Ordering<T> {
        return new Ordering(
            (left: T, right: T) => {
                const thisOrder = this.run(left, right)
                if (thisOrder === 0) {
                    return next.run(left, right)
                } else {
                    return thisOrder
                }
            }
        )
    }
    
    static natural<A>(): Ordering<A> { 
        return new Ordering((l: A, r: A) => {
            if (l < r) {
                return -1
            } else if (l > r) {
                return 1
            } else {
                return 0
            }
        })
    }
    
    undefinedLast(): Ordering<T | undefined> {
        return new Ordering((l?: T, r?: T) => {
            const undefL = l === undefined
            const undefR = r === undefined
            if (undefL && undefR) {
                return 0
            } else if (undefR) {
                return -1
            } else if (undefL) {
                return 1
            } else {
                return this.run(l, r)
            }
        })
    }

    coMap<A>(f: (x: A) => T): Ordering<A> {
        return new Ordering((left: A, right: A) => this.run(f(left), f(right)))
    }
}

In [17]:
const compareExactLanguageLevel = (language: string): Ordering<Interpreter> => Ordering
    .natural()
    .undefinedLast()
    .coMap((i: Interpreter) => i.languages.find(l => l.languageName === language)?.level)
    
const compareApproxLanguageLevel = (language: string): Ordering<Interpreter> => Ordering
    .natural()
    .undefinedLast()
    .coMap((i: Interpreter) => i.languages.find(l => l.languageName.includes(language))?.level)

const order = compareExactLanguageLevel('french').then(compareApproxLanguageLevel('french')).run
    
show(EXAMPLE_DATA.slice().sort(order))

[
  {
    "languages": [
      {
        "languageName": "french",
        "level": 3
      }
    ]
  },
  {
    "languages": [
      {
        "languageName": "french (québécois)",
        "level": 1
      },
      {
        "languageName": "italian",
        "level": 2
      }
    ]
  },
  {
    "languages": [
      {
        "languageName": "italian",
        "level": 3
      }
    ]
  }
]
